In [1]:
import pandas as pd ## Обновить pandas до последней версии
import numpy as np
import math
import os
import glob
import mysql.connector
import datetime
from mysql.connector import errorcode
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
from datetime import date
import string

from os import listdir
from os.path import isfile, join


import warnings
warnings.filterwarnings("ignore")

In [2]:
user = 'project_user'
database = 'project_db'

In [3]:
my_path = 'C:\\Users\\User\\Documents\\Python Scripts\\IT\\Untitled Folder' 

#### Блок Калинина Артёма

In [4]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

payments_table = pd.read_sql_query("SELECT * FROM payments", cnx)
payments_table.keys()

Index(['contract_number', 'payment_date', 'amount_due', 'amount_paid'], dtype='object')

In [5]:
def get_90_day_default(contract_number):
    table = payments_table[payments_table['contract_number'] == contract_number]## Выбираем строки с конкретным номером контракта
    table = table.reset_index() #обнуляем индексы, чтобы при дальнейшей итерации всё работало с 0
    for i in  range(0, len(table['amount_paid'])):
        try:
            if (table['amount_paid'][i] < table['amount_due'][i] and table['amount_paid'][i+1] < table['amount_due'][i+1] and
                table['amount_paid'][i+2] < table['amount_due'][i+2] and table['amount_paid'][i+3] < table['amount_due'][i+3]):
                #Проверяем чтобы заёмщик не платил 4 даты подряд
                date_90 = table['payment_date'][i + 3] #фиксируем дату просрочки
                date_90 = (str(pd.to_datetime(date_90).year) + '-' + str(pd.to_datetime(date_90).month) + 
                           '-' + str(pd.to_datetime(date_90).day)) #переводим в нужный формат
                break
                
            else:
                date_90 = '0000-00-00' #если нет просрочки то передаём такую дату
        except:
            True
    return date_90

In [12]:
data = []
for i in payments_table['contract_number'].unique():
    if get_90_day_default(i) == '0000-00-00':
        data.append((i, get_90_day_default(i), 0))
    else:
        data.append((i, get_90_day_default(i), 1))
data

[(100524, '0000-00-00', 0),
 (100552, '0000-00-00', 0),
 (100493, '0000-00-00', 0),
 (100797, '0000-00-00', 0),
 (100929, '0000-00-00', 0),
 (100022, '0000-00-00', 0),
 (100127, '0000-00-00', 0),
 (100520, '0000-00-00', 0),
 (100190, '0000-00-00', 0),
 (100600, '0000-00-00', 0),
 (100932, '0000-00-00', 0),
 (100582, '0000-00-00', 0),
 (100990, '0000-00-00', 0),
 (100869, '0000-00-00', 0),
 (100614, '0000-00-00', 0),
 (100323, '0000-00-00', 0),
 (100997, '0000-00-00', 0),
 (100765, '0000-00-00', 0),
 (100122, '0000-00-00', 0),
 (100068, '0000-00-00', 0),
 (100414, '0000-00-00', 0),
 (100610, '0000-00-00', 0),
 (100703, '0000-00-00', 0),
 (100684, '2017-12-14', 1),
 (100620, '0000-00-00', 0),
 (100506, '0000-00-00', 0),
 (100176, '0000-00-00', 0),
 (100431, '0000-00-00', 0),
 (100206, '0000-00-00', 0),
 (100851, '0000-00-00', 0),
 (100743, '0000-00-00', 0),
 (100518, '0000-00-00', 0),
 (100211, '0000-00-00', 0),
 (100476, '0000-00-00', 0),
 (100321, '0000-00-00', 0),
 (100297, '0000-00-0

In [24]:
cnx = mysql.connector.connect(host = 'localhost', database = database, user = user)
cursor = cnx.cursor(buffered=True)

cursor.execute("""ALTER TABLE contracts ADD default_ INT """)

for i in data:
    cursor.execute("""UPDATE contracts SET default_ = %d WHERE contract_number = %d """ %(i[2], i[0]))

#'0000-00-00' Означает что нет просрочки

cnx.commit()

In [27]:
applications_defaults = pd.read_sql_query("""SELECT age, age_of_car, children, education, employed_by, family, 
gender, house_ownership, housing, income, income_type, marital_status, position, default_
FROM applications
INNER JOIN contracts ON
contracts.id = applications.id;""", cnx)
applications_defaults.head()

,age,age_of_car,children,education,employed_by,family,gender,house_ownership,housing,income,income_type,marital_status,position,default_
0,40,17.0,1,0,1,3,0,1,0,171000,0,1,1,0
1,50,NaN,0,2,1,2,1,1,0,202500,0,1,6,1
2,40,14.0,1,2,20,3,0,1,0,202500,0,1,10,0
3,60,NaN,0,2,25,1,0,0,1,67500,1,3,0,0
4,33,NaN,1,0,9,2,0,1,0,90000,2,3,4,0


In [ ]:
defaults = applications_defaults.default_
defaults

In [31]:
applications_defaults = applications_defaults.drop(['default_'], axis = 1)

### Правила обработки информации:
 1. Каждой категории текстовых полей будет присваиваться балл
 2. Непрерывные переменные сначала группируются по 25% выборки

In [256]:
defaults = applications_defaults.default_
defaults

AttributeError: 'DataFrame' object has no attribute 'default_'

In [367]:
max_bin = 20
force_bin = 4

# define a binning function
def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def woe_iv_categorical(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            conv = char_bin(target, df1[i])
            conv["VAR_NAME"] = i
            count = count + 1
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [403]:
applications_defaults_1  = applications_defaults.copy()

In [404]:
applications_defaults_1['age'].describe()

count    60.000000
mean     45.316667
std      12.155060
min      21.000000
25%      34.750000
50%      46.500000
75%      54.250000
max      71.000000
Name: age, dtype: float64

In [405]:
applications_defaults_1['age_of_car'].describe()

count    16.000000
mean     10.312500
std       7.551766
min       1.000000
25%       3.000000
50%       9.000000
75%      14.750000
max      26.000000
Name: age_of_car, dtype: float64

In [406]:
applications_defaults_1['income'].describe()

count        60.000000
mean     171677.816667
std      113656.314964
min       38419.000000
25%       96750.000000
50%      126000.000000
75%      202500.000000
max      540000.000000
Name: income, dtype: float64

In [407]:
applications_defaults_1['age'] = np.where( (applications_defaults_1['age'] <= 34) & (applications_defaults_1['age'] >= 21), 0, 
                               np.where ((applications_defaults_1['age'] <= 46) & (applications_defaults_1['age'] > 34), 1,
                                        np.where ((applications_defaults_1['age'] <= 54) & (applications_defaults_1['age'] > 46), 2, 3)))

In [408]:
applications_defaults_1['income'] = np.where( (applications_defaults_1['income'] <= 96750) & (applications_defaults_1['income'] >= 38419), 0, 
                               np.where ((applications_defaults_1['income'] <= 126000) & (applications_defaults_1['income'] > 96750), 1,
                                        np.where ((applications_defaults_1['income'] <= 202500) & (applications_defaults_1['income'] > 126000), 2, 3)))

In [409]:
applications_defaults_1['age_of_car'] = np.where( (applications_defaults_1['age_of_car'] <= 3) & (applications_defaults_1['age_of_car'] >= 1), 0, 
                               np.where ((applications_defaults_1['age_of_car'] <= 9) & (applications_defaults_1['age_of_car'] > 3), 1,
                                        np.where ((applications_defaults_1['age_of_car'] <= 15) & (applications_defaults_1['age_of_car'] > 9), 2, 3)))

In [410]:
applications_defaults_1.head(5)

,age,age_of_car,children,education,employed_by,family,gender,house_ownership,housing,income,income_type,marital_status,position
0,1,3,1,0,1,3,0,1,0,2,0,1,1
1,2,3,0,2,1,2,1,1,0,2,0,1,6
2,1,2,1,2,20,3,0,1,0,2,0,1,10
3,3,3,0,2,25,1,0,0,1,0,1,3,0
4,0,3,1,0,9,2,0,1,0,0,2,3,4


In [411]:
woe_iv_cat_df = woe_iv_categorical(applications_defaults_1, defaults)[0]
woe_iv_cat_df

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,age,0,0,15,0,0.000000,15,1.000000,0.0,0.272727,0.000000,0.412556
1,age,1,1,15,1,0.066667,14,0.933333,0.2,0.254545,-0.241162,0.412556
2,age,2,2,15,3,0.200000,12,0.800000,0.6,0.218182,1.011601,0.412556
3,age,3,3,15,1,0.066667,14,0.933333,0.2,0.254545,-0.241162,0.412556
4,age_of_car,0,0,5,1,0.200000,4,0.800000,0.2,0.072727,1.011601,0.128749
5,age_of_car,1,1,3,0,0.000000,3,1.000000,0.0,0.054545,0.000000,0.128749
6,age_of_car,2,2,4,0,0.000000,4,1.000000,0.0,0.072727,0.000000,0.128749
7,age_of_car,3,3,48,4,0.083333,44,0.916667,0.8,0.800000,0.000000,0.128749
8,children,0,0,47,5,0.106383,42,0.893617,1.0,0.763636,0.269664,0.063739
9,children,1,1,12,0,0.000000,12,1.000000,0.0,0.218182,0.000000,0.063739


### Правило присвоения баллов: 
 1. Ранжируем категории в каждой переменной по возрастанию WOE
 2. Самому низкому WOE назначается балл равный одному - далее по возрастанию WOE каждой категории добавляется три балла
 3. Полученные таким образом баллы нормализуются к шкале от 1 до 100: количество баллов конкретной категории делится на сумму уникальных баллов и умножается на 100%

In [416]:
def get_scores(var_name):
    count = 0
    values = [0]
    for i in range(0, woe_iv_cat_df[woe_iv_cat_df['VAR_NAME'] == var_name].sort_values(['WOE']).shape[0] - 1):
        table = woe_iv_cat_df[woe_iv_cat_df['VAR_NAME'] == var_name].sort_values(['WOE'])
        if table.iloc[i][10] == table.iloc[i + 1][10]:
            values.append(count)
        else:
            count = count + 3
            values.append(count)
    c = table
    c['scores'] = values
    return c

In [417]:
new_table = pd.DataFrame()
for i in woe_iv_cat_df['VAR_NAME'].unique():
    new_table = new_table.append(get_scores(i))

In [418]:
our_dict = []
for i in new_table['VAR_NAME'].unique():
    old_keys = new_table[new_table['VAR_NAME'] == i].MIN_VALUE
    new_scores = new_table[new_table['VAR_NAME'] == i].scores
    dictionary = dict(zip(old_keys, new_scores))
    our_dict.append((i, dictionary))

In [419]:
our_dict

[('age', {1: 0, 3: 0, 0: 3, 2: 6}),
 ('age_of_car', {1: 0, 2: 0, 3: 0, 0: 3}),
 ('children', {1: 0, 2: 0, 0: 3}),
 ('education', {0: 0, 1: 3, 2: 6}),
 ('employed_by',
  {0: 0,
   23: 0,
   20: 0,
   19: 0,
   18: 0,
   16: 0,
   15: 0,
   14: 0,
   13: 0,
   24: 0,
   11: 0,
   12: 0,
   9: 0,
   8: 0,
   7: 0,
   6: 0,
   5: 0,
   4: 0,
   3: 0,
   2: 0,
   10: 0,
   25: 3,
   17: 6,
   1: 9,
   21: 12}),
 ('family', {3: 0, 4: 0, 2: 3, 1: 6}),
 ('gender', {1: 0, 0: 3}),
 ('house_ownership', {0: 0, 1: 3}),
 ('housing', {1: 0, 2: 0, 3: 0, 0: 3}),
 ('income', {3: 0, 0: 3, 2: 6, 1: 9}),
 ('income_type', {2: 0, 3: 3, 1: 6, 0: 9}),
 ('marital_status', {1: 0, 2: 3, 3: 3, 0: 6, 4: 9}),
 ('position',
  {0: 0, 1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 7: 3, 8: 3, 9: 3, 11: 3, 6: 6, 10: 9})]

In [420]:
for i in our_dict:
    applications_defaults_1[i[0]] = applications_defaults_1[i[0]].map(i[1] )
applications_defaults_1.head()

,age,age_of_car,children,education,employed_by,family,gender,house_ownership,housing,income,income_type,marital_status,position
0,0,0,0,0,9,0,3,3,3,6,9,0,3
1,6,0,3,6,9,3,0,3,3,6,9,0,6
2,0,0,0,6,0,0,3,3,3,6,9,0,9
3,0,0,3,6,3,6,3,0,0,3,6,3,0
4,3,0,0,0,0,3,3,3,3,3,0,3,3


In [421]:
for i in applications_defaults_1:
    applications_defaults_1[i] = [(x / applications_defaults_1[i].unique().sum() * 100) for x in applications_defaults_1[i]]

In [423]:
applications_defaults_1

,age,age_of_car,children,education,employed_by,family,gender,house_ownership,housing,income,income_type,marital_status,position
0,0.000000,0.0,0.0,0.000000,30.0,0.000000,100.0,100.0,100.0,33.333333,50.000000,0.000000,16.666667
1,66.666667,0.0,100.0,66.666667,30.0,33.333333,0.0,100.0,100.0,33.333333,50.000000,0.000000,33.333333
2,0.000000,0.0,0.0,66.666667,0.0,0.000000,100.0,100.0,100.0,33.333333,50.000000,0.000000,50.000000
3,0.000000,0.0,100.0,66.666667,10.0,66.666667,100.0,0.0,0.0,16.666667,33.333333,16.666667,0.000000
4,33.333333,0.0,0.0,0.000000,0.0,33.333333,100.0,100.0,100.0,16.666667,0.000000,16.666667,16.666667
5,66.666667,0.0,100.0,0.000000,0.0,33.333333,0.0,100.0,100.0,0.000000,0.000000,0.000000,16.666667
6,33.333333,0.0,100.0,0.000000,30.0,66.666667,0.0,100.0,0.0,16.666667,16.666667,16.666667,33.333333
7,0.000000,0.0,100.0,66.666667,0.0,66.666667,0.0,100.0,100.0,33.333333,16.666667,16.666667,33.333333
8,33.333333,0.0,100.0,66.666667,0.0,33.333333,100.0,100.0,100.0,50.000000,16.666667,33.333333,16.666667
9,33.333333,0.0,100.0,0.000000,20.0,33.333333,100.0,0.0,100.0,50.000000,16.666667,0.000000,50.000000


### Выгрузка из этого блока

In [424]:
applications_defaults_1.to_csv(my_path + '\\output_3\\scores_for_vars.csv', header=True)
woe_iv_cat_df.to_csv(my_path + '\\output_3\\woe_iv_table.csv', header=True)
